<a href="https://colab.research.google.com/github/greg3511/generative_ai/blob/main/Week9_AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_AI%E5%81%B5%E6%8E%A2Two_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [ ]:
!pip install aisuite[all]

### 2. 使用 AISuite 的準備

In [4]:
import aisuite as ai

In [14]:
# provider_planner = "groq"
# model_planner="llama3-70b-8192"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

provider_planner = "groq"
model_planner = "gemma2-9b-it"

# provider_writer = "groq"
# model_writer = "gemma2-9b-it"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [13]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [6]:
system_planner = "請用台灣的中文回應，你是一個台灣有名的偵探的助手，你擅長幫助偵探假設案件發生的原因，你會根據得到的資訊做出多種可能的假設。"
system_writer = "請用台灣的中文回應，你是一個台灣有名的偵探，你的助手會幫你假設案件可能發生的原因，你擅長從這些假設中選擇最有可能發生的原因。"

In [24]:
def lucky_post(prompt):
    # Step 1: CoT - 思考五種幸運理由
    planning_prompt = f"使用者說：{prompt}。請幫我假設五種案件發生的原因。請用台灣的中文回應。"
    lucky_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我想到的五個假設：\n{lucky_reasons}\n\n請從中選出一個最有可能的假設，然後根據這個假設推理案發原因。請用台灣的中文回應。"
    final_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, final_post

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [8]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [21]:
import gradio as gr

In [25]:
with gr.Blocks() as demo:
    gr.Markdown("### AI偵探")
    gr.Markdown("請描述一個案發現場，配上一些細節，讓AI偵探推理出原因。")
    user_input = gr.Textbox(label="發生了什麼?")
    btn = gr.Button("生成AI偵探的推理")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個可能的假設（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終結論（Po 文）")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://988493a057efc1d5b1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
